In [1]:
import pandas as pd
pd.set_option('display.max_columns', 100)

In [2]:
spark.sql('use covid_2020_q3')

DataFrame[]

In [ ]:
### Check Cardiac arrest instances - D_ICD10_I46.9

#spark.sql('select * from LR_Dec.LR_inhosp_mort_diag_tmp where covar="D_ICD10_I46.9"limit 10').toPandas()
##### some on early dates but mostly on 2020
##check patients "bdafda7b", "1faa8e09"
spark.sql('select * from LR_Dec.LR_inhosp_mort_diag_tmp where personid in ("bdafda7b", "1faa8e09") order by personid,enc_date desc').toPandas()
spark.sql('select * from LR_Dec.LR_cohort_outcomes3 where personid in ("bdafda7b", "1faa8e09") ').toPandas()
spark.sql('select * from demographics_corrected where personid in ("bdafda7b", "1faa8e09") ').toPandas()

----- So as conclusion from above those patient with Cardiac arrest codes, survive :) , data is ok so far

In [1]:
#### Create data dictionary for interpretations
#types_d_rev[575+1] ### 'D_ICD10_I46.9' cardiac arrest
#types_d_rev[53022+1] ### 'L_LOINC_19928-1$Nonrebreather oxygen mask'
#types_d_rev[49094+1] ### 'M_MultumdNUM_d00015' ibuprofen
#types_d_rev[51761+1] ### M_MultumCat_61

In [4]:
# Diagnosis
spark.sql('create table lr_dec.d_diag as select distinct codetype, conditioncode, condition, concat("D_",(case when codetype like ("%ICD%10%") then "ICD10" when codetype like ("%ICD%9%D%") then "ICD9" when codetype like ("%SNOMED%") then "SNOMED" end),"_",conditioncode) covar from condition where (codetype like ("%ICD%10%") or codetype like ("%ICD%9%D%") or codetype like ("%SNOMED%"))').toPandas()
# Medication
spark.sql(' create table lr_dec.d_med as select * from (select distinct drug_id med_id, drug_desc med_desc, concat("M_MultumdNUM_",lower(trim(drug_id))) covar from multum_dnum_descript union select distinct multum_category_id med_id, category_name med_desc, concat("M_MultumCat_",lower(trim(multum_category_id))) covar from  multum_drug_categories)x').toPandas()
#Procedures
spark.sql('create table LR_Dec.d_proc as  select distinct codetype,procedurecode, procedure, concat("P_",(case when codetype like ("%CPT-4%") then "CPT4" when codetype like ("%ICD%9%P%") then "ICD9PCS" when codetype like ("%ICD%10%P%") then "ICD10PCS" when codetype like ("HCPCS%") then "HCPCS" when codetype like ("%SNOMED%") then "SNOMED" else upper(trim(codetype)) end),"_",upper(trim(procedurecode))) covar from procedure d where (codetype in ("ICD-10-PCS","HCPCS","ICD-9-CM Procedures","HCPCS Level II","CPT-4 (HCPCS Level I)","LOINC","SNOMED CT")) ')
# Demographics
spark.sql('create table lr_dec.d_demo as select distinct "gender" category, d.gender cat_value, concat("g_",( case when lower(trim(d.gender)) in ("female","male") then lower(trim(d.gender)) else "other"  end))covar from demographics_corrected d '
         +' union select distinct "age" category, age_at_encounter cat_value, concat("a_",cast( age_at_encounter as string))covar from lr_dec.LR_inhosp_pt_dts_encs '
         +' union select distinct "gender" category,race cat_value, concat("r_",( case when lower(race) like "%white%" then "1"  when lower(race) like "%african%american%" then "2" when lower(race) like "%asian%" then "3" when lower(race) like "%alaska%native%" then "4" else "5"  end))covar from demographics_corrected d  '
         +' union select distinct "ethnicity" category, ethnicity cat_value, "e_hispanic" covar  from demographics_corrected d where trim(ethnicity)="Hispanic or Latino" '
         +' union select distinct "censusRegion" category, censusRegion cat_value, concat("cr_",substring(trim(censusRegion),1,1))covar from tenant t  '
         +' union select distinct "censusDivision" category, censusDivision cat_value, concat("cd_",substring(trim(censusDivision),1,1))covar from tenant t '
         ).toPandas()


""


In [11]:
spark.sql ('create table lr_dec.d_lab_tmp as select distinct codetype,resultcode,result from result where codetype like "L%" or codetype like "S%" or codetype like "N%"').toPandas()
spark.sql ('create table lr_dec.d_lab_tmp_cod1 as select distinct codifiedvaluecodetype,codifiedvaluecode,codifiedvalue from result where (codetype like "L%" or codetype like "S%" or codetype like "N%") and resulttype="CODIFIED" and codifiedvalue is not null').toPandas()


""


In [ ]:
#### Below are sample of dictionaries

In [4]:
spark.sql ('select * from lr_dec.d_lab_tmp limit 5').toPandas()

,codetype,resultcode,result
0,SNOMED CT,425014005,Substance use education
1,LOINC,2157-6,Creatine kinase [Enzymatic activity/volume] in...
2,LOINC,35012-4,Segmented neutrophils [#/volume] in Pleural fluid
3,LOINC,13317-3,Methicillin resistant Staphylococcus aureus [P...
4,LOINC,19057-9,ABO and Rh group [Type] in Blood from Newborn


In [5]:
spark.sql ('select * from lr_dec.d_lab_tmp_cod1 limit 5').toPandas()

,codifiedvaluecodetype,codifiedvaluecode,codifiedvalue
0,SNOMED CT,732769006,5.5
1,SNOMED CT,34691000175105,Diabetic diet
2,SNOMED CT,300686005,Difficulty managing personal financial activities
3,SNOMED CT,449781000124107,Prepared
4,SNOMED CT,423475008,Heart failure education


In [11]:
spark.sql('select * from lr_dec.d_diag where covar="D_ICD10_I46.9"').toPandas()

,codetype,conditioncode,condition,covar
0,ICD-10-CM,I46.9,"Cardiac arrest, cause unspecified",D_ICD10_I46.9


In [31]:
spark.sql('select substring(covar,3) new_covar,covar,"diagnosis" category, codetype , conditioncode code , condition desc from lr_dec.d_diag limit 5 ').toPandas()

,new_covar,covar,category,codetype,code,desc
0,ICD10_E11.621,D_ICD10_E11.621,diagnosis,ICD-10-CM,E11.621,Type 2 diabetes mellitus with foot ulcer
1,ICD10_F19.929,D_ICD10_F19.929,diagnosis,ICD-10-CM,F19.929,"Other psychoactive substance use, unspecified ..."
2,ICD10_G56.01,D_ICD10_G56.01,diagnosis,ICD-10-CM,G56.01,"Carpal tunnel syndrome, right upper limb"
3,ICD9_723.1,D_ICD9_723.1,diagnosis,ICD-9-CM Diagnoses,723.1,Cervicalgia
4,ICD10_D51.0,D_ICD10_D51.0,diagnosis,ICD-10-CM,D51.0,Vitamin B12 deficiency anemia due to intrinsic...


In [10]:
spark.sql('select * from lr_dec.d_proc limit 5').toPandas()

,codetype,procedurecode,procedure,covar
0,HCPCS Level II,J0696,"Injection, ceftriaxone sodium, per 250 mg",P_HCPCS_J0696
1,ICD-10-PCS,0B110F4,Bypass Trachea to Cutaneous with Tracheostomy ...,P_ICD10PCS_0B110F4
2,ICD-10-PCS,06HN33Z,Insertion of Infusion Device into Left Femoral...,P_ICD10PCS_06HN33Z
3,SNOMED CT,85562004,Hand structure,P_SNOMED_85562004
4,SNOMED CT,8613002,Operation on appendix,P_SNOMED_8613002


In [30]:
spark.sql('select substring(covar,3) new_covar,covar,"procedure" category, codetype , procedurecode code , procedure desc from lr_dec.d_proc limit 5').toPandas()

,new_covar,covar,category,codetype,code,desc
0,HCPCS_J0696,P_HCPCS_J0696,procedure,HCPCS Level II,J0696,"Injection, ceftriaxone sodium, per 250 mg"
1,ICD10PCS_0B110F4,P_ICD10PCS_0B110F4,procedure,ICD-10-PCS,0B110F4,Bypass Trachea to Cutaneous with Tracheostomy ...
2,ICD10PCS_06HN33Z,P_ICD10PCS_06HN33Z,procedure,ICD-10-PCS,06HN33Z,Insertion of Infusion Device into Left Femoral...
3,SNOMED_85562004,P_SNOMED_85562004,procedure,SNOMED CT,85562004,Hand structure
4,SNOMED_8613002,P_SNOMED_8613002,procedure,SNOMED CT,8613002,Operation on appendix


In [3]:
spark.sql('select * from lr_dec.d_med limit 10').toPandas()

,med_id,med_desc,covar
0,d00303,miSOPROStol,M_MultumdNUM_d00303
1,d00844,sodium thiosalicylate,M_MultumdNUM_d00844
2,d00943,phensuximide,M_MultumdNUM_d00943
3,d01256,etretinate,M_MultumdNUM_d01256
4,d03511,phenazopyridine-sulfamethoxazole,M_MultumdNUM_d03511
5,d04175,diphenhydramine-magnesium salicylate,M_MultumdNUM_d04175
6,d04523,ubiquinone,M_MultumdNUM_d04523
7,d04719,coagulation factor VIIa,M_MultumdNUM_d04719
8,d04753,travoprost ophthalmic,M_MultumdNUM_d04753
9,d04796,perflutren,M_MultumdNUM_d04796


In [34]:
spark.sql('select substring(covar,3) new_covar,covar,"Medication" category,split(covar,"_")[1] codetype , med_id code , med_desc desc from lr_dec.d_med where covar like "M_MultumCat%" limit 5 ').toPandas()

,new_covar,covar,category,codetype,code,desc
0,MultumCat_53,M_MultumCat_53,Medication,MultumCat,53,vasodilators
1,MultumCat_347,M_MultumCat_347,Medication,MultumCat,347,gamma-aminobutyric acid analogs
2,MultumCat_119,M_MultumCat_119,Medication,MultumCat,119,vitamins
3,MultumCat_153,M_MultumCat_153,Medication,MultumCat,153,plasma expanders
4,MultumCat_293,M_MultumCat_293,Medication,MultumCat,293,glucose elevating agents


In [35]:
spark.sql('select covar new_covar,covar,"demographics" category,category codetype , cat_value code , cat_value desc from lr_dec.d_demo limit 5 ').toPandas()

,new_covar,covar,category,codetype,code,desc
0,a_42,a_42,demographics,age,42,42
1,g_other,g_other,demographics,gender,Unknown,Unknown
2,a_69,a_69,demographics,age,69,69
3,a_28,a_28,demographics,age,28,28
4,r_4,r_4,demographics,gender,American Indian or Alaska Native,American Indian or Alaska Native


In [3]:
spark.sql('select * from lr_dec.d_demo limit 5').toPandas()

,category,cat_value,covar
0,age,42,a_42
1,gender,Unknown,g_other
2,age,69,a_69
3,age,28,a_28
4,gender,American Indian or Alaska Native,r_4


In [ ]:
### need to check if loinc and SNOMED CT same code appear in different tables and if they have the same meanings

In [4]:
spark.sql(' select distinct d.codetype,* from lr_dec.d_diag d, lr_dec.d_lab_tmp l, lr_dec.d_proc p where d.codetype=l.codetype and d.codetype=p.codetype').toPandas()

,codetype
0,SNOMED CT


In [5]:
spark.sql(' select distinct d.codetype,* from lr_dec.d_diag d, lr_dec.d_proc p where d.codetype=p.codetype and conditioncode=procedurecode').toPandas()
####3063 code

,codetype,codetype,conditioncode,condition,covar,codetype,procedurecode,procedure,covar
0,SNOMED CT,SNOMED CT,92824003,"Neurofibromatosis, type 1",D_SNOMED_92824003,SNOMED CT,92824003,"Neurofibromatosis, type 1",P_SNOMED_92824003
1,SNOMED CT,SNOMED CT,95344007,Ulcer of lower extremity,D_SNOMED_95344007,SNOMED CT,95344007,Ulcer of lower extremity,P_SNOMED_95344007
2,SNOMED CT,SNOMED CT,26929004,"Alzheimer's disease, NOS",D_SNOMED_26929004,SNOMED CT,26929004,Alzheimer's disease,P_SNOMED_26929004
3,SNOMED CT,SNOMED CT,698378009,Coronary artery bypass graft operation planned,D_SNOMED_698378009,SNOMED CT,698378009,Coronary artery bypass graft operation planned,P_SNOMED_698378009
4,SNOMED CT,SNOMED CT,423463003,Basal cell carcinoma of back,D_SNOMED_423463003,SNOMED CT,423463003,Basal cell carcinoma of back,P_SNOMED_423463003
...,...,...,...,...,...,...,...,...,...
3058,SNOMED CT,SNOMED CT,416098002,Drug allergy,D_SNOMED_416098002,SNOMED CT,416098002,Allergy to drug,P_SNOMED_416098002
3059,SNOMED CT,SNOMED CT,236105003,Oesophageal anastomotic stricture,D_SNOMED_236105003,SNOMED CT,236105003,Oesophageal anastomotic stricture,P_SNOMED_236105003
3060,SNOMED CT,SNOMED CT,283359004,Laceration of forehead,D_SNOMED_283359004,SNOMED CT,283359004,Laceration of forehead,P_SNOMED_283359004
3061,SNOMED CT,SNOMED CT,83746006,Chronic schizophrenia,D_SNOMED_83746006,SNOMED CT,83746006,Chronic schizophrenia,P_SNOMED_83746006


In [6]:
spark.sql(' select distinct l.codetype from lr_dec.d_lab_tmp l, lr_dec.d_proc p where l.codetype=p.codetype').toPandas()

,codetype
0,LOINC
1,SNOMED CT


In [7]:
spark.sql(' select distinct l.codetype,* from lr_dec.d_lab_tmp l, lr_dec.d_proc p where l.codetype=p.codetype and resultcode=procedurecode').toPandas()
### nice 452

,codetype,codetype,resultcode,result,codetype,procedurecode,procedure,covar
0,SNOMED CT,SNOMED CT,267038008,Edema,SNOMED CT,267038008,Oedema,P_SNOMED_267038008
1,SNOMED CT,SNOMED CT,266257000,Transient cerebral ischaemia,SNOMED CT,266257000,Transient cerebral ischaemia,P_SNOMED_266257000
2,SNOMED CT,SNOMED CT,368209003,Right upper arm structure,SNOMED CT,368209003,Right upper arm structure,P_SNOMED_368209003
3,SNOMED CT,SNOMED CT,6685009,Structure of right ankle,SNOMED CT,6685009,Structure of right ankle,P_SNOMED_6685009
4,SNOMED CT,SNOMED CT,405729008,Haematochezia,SNOMED CT,405729008,Hematochezia,P_SNOMED_405729008
...,...,...,...,...,...,...,...,...
447,SNOMED CT,SNOMED CT,6471006,Suicidal ideation,SNOMED CT,6471006,Suicidal thoughts,P_SNOMED_6471006
448,SNOMED CT,SNOMED CT,309211008,Gastric biopsy sample,SNOMED CT,309211008,Gastric biopsy sample,P_SNOMED_309211008
449,SNOMED CT,SNOMED CT,161891005,Backache,SNOMED CT,161891005,Backache,P_SNOMED_161891005
450,LOINC,LOINC,94500-6,SARS coronavirus 2 RNA [Presence] in Respirato...,LOINC,94500-6,SARS-CoV-2 (COVID-19) RNA [Presence] in Respir...,P_LOINC_94500-6


##### I believe the right decision is to remove the first prefix D_ P_ except for demographics, the second is more indicative to the type

In [11]:
spark.sql(' select distinct *, substring(replace(covar,".",""), 3) new_covar from lr_dec.d_diag limit 10').toPandas()

,codetype,conditioncode,condition,covar,new_covar
0,SNOMED CT,301826004,Intermittent stridor,D_SNOMED_301826004,SNOMED_301826004
1,ICD-10-CM,E72.59,Other disorders of glycine metabolism,D_ICD10_E72.59,ICD10_E7259
2,SNOMED CT,365574009,Life event finding,D_SNOMED_365574009,SNOMED_365574009
3,ICD-10-CM,N44.1,Cyst of tunica albuginea testis,D_ICD10_N44.1,ICD10_N441
4,ICD-10-CM,I75.021,Atheroembolism of right lower extremity,D_ICD10_I75.021,ICD10_I75021
5,SNOMED CT,5148006,Hypertensive heart disease with congestive hea...,D_SNOMED_5148006,SNOMED_5148006
6,ICD-10-CM,E77.0,Defects in post-translational modification of ...,D_ICD10_E77.0,ICD10_E770
7,ICD-9-CM Diagnoses,719.49,"Pain in joint, multiple sites",D_ICD9_719.49,ICD9_71949
8,ICD-9-CM Diagnoses,729.81,Swelling of limb,D_ICD9_729.81,ICD9_72981
9,ICD-10-CM,K43.3,"Parastomal hernia with obstruction, without ga...",D_ICD10_K43.3,ICD10_K433


In [ ]:
### Counts

In [1]:
spark.sql('select codetype, count(distinct covar) from lr_dec.d_proc group by codetype').toPandas()

,codetype,count(DISTINCT covar)
0,ICD-9-CM Procedures,1825
1,HCPCS Level II,1211
2,CPT-4 (HCPCS Level I),7030
3,ICD-10-PCS,16543
4,LOINC,105
5,HCPCS,18
6,SNOMED CT,14318


In [2]:
spark.sql('select codetype, count(distinct covar) from lr_dec.d_diag group by codetype').toPandas()

,codetype,count(DISTINCT covar)
0,ICD-10-CM,37492
1,ICD-9-CM Diagnoses,8742
2,ICD-10,1
3,ICD-10-PCS,3
4,ICD-10,9
5,SNOMED CT,11984


In [ ]:
### Counts in our cohort

In [4]:
spark.sql('select codetype, count(distinct covar) from lr_dec.d_proc  where covar in (select distinct covar from LR_Dec.LR_inhosp_mort_proc_tmp) group by codetype').toPandas()

,codetype,count(DISTINCT covar)
0,ICD-9-CM Procedures,1611
1,HCPCS Level II,1044
2,CPT-4 (HCPCS Level I),6255
3,ICD-10-PCS,13612
4,LOINC,77
5,HCPCS,15
6,SNOMED CT,10560


In [5]:
spark.sql('select codetype, count(distinct covar) from lr_dec.d_diag where covar in (select distinct covar from LR_Dec.LR_inhosp_mort_diag_tmp) group by codetype').toPandas()


,codetype,count(DISTINCT covar)
0,ICD-10-CM,31830
1,ICD-9-CM Diagnoses,7789
2,ICD-10,1
3,ICD-10,5
4,SNOMED CT,9453


In [10]:
spark.sql('select count (distinct covar) from lr_dec.d_diag').toPandas()### 58226 unique covar and 59608 records

,count(DISTINCT covar)
0,58226


In [13]:
spark.sql('show tables').toPandas()

,database,tableName,isTemporary
0,covid_2020_q3,allergy,False
1,covid_2020_q3,allergy_reaction,False
2,covid_2020_q3,condition,False
3,covid_2020_q3,covid_labs,False
4,covid_2020_q3,demographics,False
5,covid_2020_q3,demographics_corrected,False
6,covid_2020_q3,encounter,False
7,covid_2020_q3,med_rec_compliance,False
8,covid_2020_q3,medication,False
9,covid_2020_q3,multum_dnum_descript,False


In [12]:
spark.sql ('select * from lr_dec.d_lab_tmp limit 5').toPandas()

,codetype,resultcode,result
0,SNOMED CT,425014005,Substance use education
1,LOINC,2157-6,Creatine kinase [Enzymatic activity/volume] in...
2,LOINC,35012-4,Segmented neutrophils [#/volume] in Pleural fluid
3,LOINC,13317-3,Methicillin resistant Staphylococcus aureus [P...
4,LOINC,19057-9,ABO and Rh group [Type] in Blood from Newborn


In [6]:
spark.sql ('select * from lr_dec.d_lab_tmp_cod1 limit 5').toPandas()

,codifiedvaluecodetype,codifiedvaluecode,codifiedvalue
0,SNOMED CT,732769006,5.5
1,SNOMED CT,34691000175105,Diabetic diet
2,SNOMED CT,300686005,Difficulty managing personal financial activities
3,SNOMED CT,449781000124107,Prepared
4,SNOMED CT,423475008,Heart failure education


In [24]:
spark.sql ('drop table lr_dec.d_lab_tmp_p1').toPandas()
spark.sql ('create table lr_dec.d_lab_tmp_p1 as select distinct substring(covar_updated,3) new_covar, covar, codetype,test_code,result cat_result,codified_value result_code from lr_dec.LR_inhosp_mort_lab_tmp_v3').toPandas()

""


In [25]:
spark.sql('select * from lr_dec.d_lab_tmp_p1 where result_code is not null limit 5').toPandas()

,new_covar,covar,codetype,test_code,cat_result,result_code
0,LOINC_44971-0$SNOMED|27751000175103,L_LOINC_44971-0$SNOMED CT_27751000175103,LOINC,44971-0,SNOMED CT,27751000175103
1,SNOMED_1421000175103$NCI|C65002,L_SNOMED CT_1421000175103$NCI_C65002,SNOMED CT,1421000175103,NCI,C65002
2,LOINC_89022-8$SNOMED|22861000175105,L_LOINC_89022-8$SNOMED CT_22861000175105,LOINC,89022-8,SNOMED CT,22861000175105
3,LOINC_7792-5$LOINC|LA15680-4,L_LOINC_7792-5$LOINC_LA15680-4,LOINC,7792-5,LOINC,LA15680-4
4,LOINC_32762-7$LOINC|LA15680-4,L_LOINC_32762-7$LOINC_LA15680-4,LOINC,32762-7,LOINC,LA15680-4


In [26]:
spark.sql ('drop table lr_dec.d_lab_tmp_p2').toPandas()
spark.sql('create table lr_dec.d_lab_tmp_p2 as select distinct a.*,b.result test_desc from lr_dec.d_lab_tmp_p1 a left join lr_dec.d_lab_tmp b on a.codetype =b.codetype and a.test_code=b.resultcode').toPandas()

""


In [27]:
spark.sql('select * from lr_dec.d_lab_tmp_p2 where result_code is not null limit 5').toPandas()

,new_covar,covar,codetype,test_code,cat_result,result_code,test_desc
0,SNOMED_49919000$SNOMED|1101000175102,L_SNOMED CT_49919000$SNOMED CT_1101000175102,SNOMED CT,49919000,SNOMED CT,1101000175102,Home safety education
1,SNOMED_6918002$LOINC|LA13035-3,L_SNOMED CT_6918002$LOINC_LA13035-3,SNOMED CT,6918002,LOINC,LA13035-3,Muscle tonus
2,LOINC_7791-7$LOINC|LA15680-4,L_LOINC_7791-7$LOINC_LA15680-4,LOINC,7791-7,LOINC,LA15680-4,Dacrocytes [Presence] in Blood by Light micros...
3,SNOMED_74964007$SNOMED|1101000175102,L_SNOMED CT_74964007$SNOMED CT_1101000175102,SNOMED CT,74964007,SNOMED CT,1101000175102,Other
4,LOINC_13945-1$SNOMED|43061000175109,L_LOINC_13945-1$SNOMED CT_43061000175109,LOINC,13945-1,SNOMED CT,43061000175109,Erythrocytes [#/area] in Urine sediment by Mic...


In [28]:
spark.sql ('drop table lr_dec.d_lab_tmp_p3').toPandas()
spark.sql('create table lr_dec.d_lab_tmp_p3 as select distinct a.*,b.codifiedvalue result_code_desc from lr_dec.d_lab_tmp_p2 a left join lr_dec.d_lab_tmp_cod1 b on a.cat_result=b.codifiedvaluecodetype and a.result_code=b.codifiedvaluecode').toPandas() 

""


In [ ]:
spark.sql('select case when upper(codetype) like "SNOMED%" then "SNOMED" else upper(codetype) end),"_",upper(test_code)) covar from lr_dec.d_lab_tmp_p3').toPandas()

In [16]:
spark.sql('select distinct new_covar, split(replace(new_covar,"$","#"),"#")[0] new_covar, split(replace(covar,"$","#"),"#")[0] covar,"Results" category,codetype, split(replace(new_covar,"$","_"),"_")[1] code, test_desc desc from lr_dec.d_lab_tmp_p3 limit 5').toPandas()

,new_covar,new_covar,covar,category,codetype,code,desc
0,LOINC_2494-3$above high normal,LOINC_2494-3,L_LOINC_2494-3,Results,LOINC,2494-3,Iodine [Mass/volume] in Serum or Plasma
1,LOINC_94500-6$not detected,LOINC_94500-6,L_LOINC_94500-6,Results,LOINC,94500-6,SARS-CoV-2 (COVID-19) RNA [Presence] in Respir...
2,LOINC_87558-3$abnormal,LOINC_87558-3,L_LOINC_87558-3,Results,LOINC,87558-3,Anaplasma phagocytophilum groEL gene [Presence...
3,SNOMED_61310001$balanced diet,SNOMED_61310001,L_SNOMED CT_61310001,Results,SNOMED CT,61310001,Nutrition education
4,LOINC_30471-7,LOINC_30471-7,L_LOINC_30471-7,Results,LOINC,30471-7,levETIRAcetam [Mass/volume] in Serum or Plasma


In [36]:
spark.sql('select new_covar,covar,"Results" category,codetype , split(new_covar,"_")[1] code , (case when result_code_desc is null then test_desc else concat(test_desc,": ",result_code_desc)end ) desc from lr_dec.d_lab_tmp_p3 where result_code is not null limit 5').toPandas()

,new_covar,covar,category,codetype,code,desc
0,SNOMED_405052004$SNOMED|33901000175105,L_SNOMED CT_405052004$SNOMED CT_33901000175105,Results,SNOMED CT,405052004$SNOMED|33901000175105,Level of stress: Less than or equal to two
1,SNOMED_6918002$LOINC|LA13035-3,L_SNOMED CT_6918002$LOINC_LA13035-3,Results,SNOMED CT,6918002$LOINC|LA13035-3,Muscle tonus
2,SNOMED_122575003$SNOMED|43121000175101,L_SNOMED CT_122575003$SNOMED CT_43121000175101,Results,SNOMED CT,122575003$SNOMED|43121000175101,Urine specimen: 10-20
3,LOINC_7791-7$LOINC|LA15680-4,L_LOINC_7791-7$LOINC_LA15680-4,Results,LOINC,7791-7$LOINC|LA15680-4,Dacrocytes [Presence] in Blood by Light micros...
4,LOINC_94500-6,L_LOINC_94500-6$urn:cerner:coding:codingsystem...,Results,LOINC,94500-6,SARS-CoV-2 (COVID-19) RNA [Presence] in Respir...


In [21]:
spark.sql('select new_covar,covar,"Results" category,codetype , split(new_covar,"_")[1] code , (case when result_code_desc is null then test_desc else concat(test_desc,": ",result_code_desc)end ) desc from lr_dec.d_lab_tmp_p3 limit 10').toPandas()


,new_covar,covar,category,codetype,code,desc
0,SNOMED_405052004$SNOMED|33901000175105,L_SNOMED CT_405052004$SNOMED CT_33901000175105,Results,SNOMED CT,405052004$SNOMED|33901000175105,Level of stress: Less than or equal to two
1,LOINC_29591-5$abnormal,L_LOINC_29591-5$Abnormal,Results,LOINC,29591-5$abnormal,Enterovirus RNA [Presence] in Unspecified spec...
2,LOINC_50405-0$anxiety,L_LOINC_50405-0$Anxiety,Results,LOINC,50405-0$anxiety,Patient symptoms [Interpretation] Narrative--a...
3,LOINC_53972-6$abnormal,L_LOINC_53972-6$Abnormal,Results,LOINC,53972-6$abnormal,Burr cells [Presence] in Urine sediment by Lig...
4,SNOMED_161086008$other: none,L_SNOMED CT_161086008$Other: none,Results,SNOMED CT,161086008$other: none,Recent countries visited
5,LOINC_13315-7,L_LOINC_13315-7$See Comment,Results,LOINC,13315-7,Bacteria identified in 24 hour Urine by Culture
6,SNOMED_704434006$orthostatic body position,L_SNOMED CT_704434006$Orthostatic body position,Results,SNOMED CT,704434006$orthostatic body position,Ability to perform personal grooming activity
7,SNOMED_364617005$absent,L_SNOMED CT_364617005$Absent,Results,SNOMED CT,364617005$absent,Fetal movement feature
8,LOINC_57408-7$above high normal,L_LOINC_57408-7$Above high normal,Results,LOINC,57408-7$above high normal,CD8+CD45RO+ cells/100 cells in Body fluid
9,SNOMED_363987008$eyeglasses,L_SNOMED CT_363987008$Eyeglasses,Results,SNOMED CT,363987008$eyeglasses,Device of eye observable


In [22]:
spark.sql('select trim(replace(substring(covar,3),".","")) new_covar,covar,"diagnosis" category, codetype , conditioncode code , condition desc from lr_dec.d_diag limit 10').toPandas()


,new_covar,covar,category,codetype,code,desc
0,ICD10_E11621,D_ICD10_E11.621,diagnosis,ICD-10-CM,E11.621,Type 2 diabetes mellitus with foot ulcer
1,ICD10_F19929,D_ICD10_F19.929,diagnosis,ICD-10-CM,F19.929,"Other psychoactive substance use, unspecified ..."
2,ICD10_G5601,D_ICD10_G56.01,diagnosis,ICD-10-CM,G56.01,"Carpal tunnel syndrome, right upper limb"
3,ICD9_7231,D_ICD9_723.1,diagnosis,ICD-9-CM Diagnoses,723.1,Cervicalgia
4,ICD10_D510,D_ICD10_D51.0,diagnosis,ICD-10-CM,D51.0,Vitamin B12 deficiency anemia due to intrinsic...
5,ICD9_36256,D_ICD9_362.56,diagnosis,ICD-9-CM Diagnoses,362.56,Macular puckering
6,ICD10_C9590,D_ICD10_C95.90,diagnosis,ICD-10-CM,C95.90,"Leukemia, unspecified not having achieved remi..."
7,ICD10_C481,D_ICD10_C48.1,diagnosis,ICD-10-CM,C48.1,Malignant neoplasm of specified parts of perit...
8,ICD10_Z0489,D_ICD10_Z04.89,diagnosis,ICD-10-CM,Z04.89,Encounter for examination and observation for ...
9,ICD10_I82C19,D_ICD10_I82.C19,diagnosis,ICD-10-CM,I82.C19,Acute embolism and thrombosis of unspecified i...


In [26]:
#spark.sql('drop table lr_dec.covar_descr')
spark.sql('create table lr_dec.covar_descr as select distinct * from ( select trim(replace(substring(covar,3),".","")) new_covar,covar,"procedure" category, codetype , procedurecode code , procedure desc from lr_dec.d_proc '
         +' union select trim(replace(substring(covar,3),".","")) new_covar,covar,"diagnosis" category, codetype , conditioncode code , condition desc from lr_dec.d_diag '
         +' union select trim(replace(substring(covar,3),".","")) new_covar,covar,"Medication" category,split(covar,"_")[1] codetype , med_id code , med_desc desc from lr_dec.d_med '
         +' union select new_covar,covar,"Results" category,codetype , split(new_covar,"_")[1] code , (case when result_code_desc is null then test_desc else concat(test_desc,": ",result_code_desc)end ) desc from lr_dec.d_lab_tmp_p3 '
         +' union select split(replace(new_covar,"$","#"),"#")[0] new_covar, split(replace(covar,"$","#"),"#")[0] covar,"Results" category,codetype, split(replace(new_covar,"$","_"),"_")[1] code, test_desc desc from lr_dec.d_lab_tmp_p3 '
         +' union select covar new_covar,covar,"demographics" category,category codetype , cat_value code , cat_value desc from lr_dec.d_demo  ) a'       
         ).toPandas()

""


In [24]:
spark.sql("select * from multum_dnum_map where drug_id like '%d07059%' ").toPandas()#d07059 10 records
spark.sql("select * from multum_dnum_descript where drug_id like '%d07059%' ").toPandas() ### No description exist (fine)

,coding_system_id,code_type,drug_code,drug_id
0,2.16.840.1.113883.6.312,Multum drug synonym,128072,d07059
1,2.16.840.1.113883.6.312,Multum drug synonym,128073,d07059
2,2.16.840.1.113883.6.312,Multum drug synonym,128086,d07059
3,2.16.840.1.113883.6.312,Multum drug synonym,128087,d07059
4,2.16.840.1.113883.6.312,Multum drug synonym,128088,d07059
5,2.16.840.1.113883.6.312,Multum drug synonym,128089,d07059
6,2.16.840.1.113883.6.312,Multum drug synonym,199752,d07059
7,2.16.840.1.113883.6.312,Multum drug synonym,199753,d07059
8,2.16.840.1.113883.6.88,RxNorm,1111261,d07059
9,2.16.840.1.113883.6.88,RxNorm,1111265,d07059


In [29]:
spark.sql('select * from lr_dec.covar_descr').toPandas().to_csv('../../Code/MedBERT/covar_desc.csv',index=False)

In [28]:
spark.sql('select * from lr_dec.covar_descr where new_covar like "%61152-5%"').toPandas()

,new_covar,covar,category,codetype,code,desc
0,LOINC_61152-5$very abnormal,L_LOINC_61152-5$Very abnormal,Results,LOINC,61152-5$very abnormal,Albumin [Mass/volume] in Serum or Plasma by Br...
1,LOINC_61152-5$normal,L_LOINC_61152-5$Normal,Results,LOINC,61152-5$normal,Albumin [Mass/volume] in Serum or Plasma by Br...
2,LOINC_61152-5$below low normal,L_LOINC_61152-5$Below low normal,Results,LOINC,61152-5$below low normal,Albumin [Mass/volume] in Serum or Plasma by Br...
3,LOINC_61152-5,L_LOINC_61152-5,Results,LOINC,61152-5,Albumin [Mass/volume] in Serum or Plasma by Br...
4,LOINC_61152-5$above high normal,L_LOINC_61152-5$Above high normal,Results,LOINC,61152-5$above high normal,Albumin [Mass/volume] in Serum or Plasma by Br...


In [18]:
spark.sql('select * from lr_dec.d_med where med_id like "%705%"').toPandas()

,med_id,med_desc,covar
0,d07054,benzalkonium chloride-menthol topical,M_MultumdNUM_d07054
1,d07055,chlorpheniramine/guaifenesin/methscopolami/PE,M_MultumdNUM_d07055
2,d04705,fludeoxyglucose F-18,M_MultumdNUM_d04705
3,d07057,nilotinib,M_MultumdNUM_d07057
4,d07705,lurasidone,M_MultumdNUM_d07705
5,d07051,ixabepilone,M_MultumdNUM_d07051
6,d03705,salicylic acid-sulfur topical,M_MultumdNUM_d03705
7,d07058,sulfacetamide sodium/sulfur/urea topical,M_MultumdNUM_d07058


In [ ]:
d